In [56]:
#pip install plotly==5.6.0

In [57]:
import plotly.express as px
import plotly.io as pio
pio.templates
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots
import pandas as pd
import datetime
from datetime import date, timedelta
import re

In [58]:
def cut(x):
    return x[:-3]

In [59]:
def cut7(x):
    return x[:10]

In [60]:
def format_date1(x):
    if x == '':
        return datetime.datetime.strptime("9999-12-31 00:00:00", "%Y-%m-%d %X").strftime("%Y-%m-%d")
    else:
        return datetime.datetime.strptime(x, "%Y-%m-%d %X").strftime("%Y-%m-%d")

In [61]:
def format_date2(x):
    if x == '' or x == 'nan':
        return datetime.datetime.strptime("9999-12-31 00:00:00", "%Y-%m-%d %X").strftime("%Y-%m-%d")
    else:
        return datetime.datetime.strptime(x, "%Y-%m-%d").strftime("%Y-%m-%d")

In [62]:
def date_converter(y):
    if y == 'nan':
        return date.fromisoformat("9999-12-31")
    else:    
        return date.fromisoformat(y)

In [63]:
# *** LOAD DATA ***
"""
track_interest_df = pd.read_csv("track_interest.csv")
subscriptions_df = pd.read_csv("subscriptions.csv")

subscriptions_sub_df = pd.DataFrame(subscriptions_df[['id', 'stripe_id', 'paid_until', 'was_trial', 'do_not_renew', 'status']])
subscriptions_sub_df = subscriptions_sub_df.rename(columns={'id': 'subscription_id'})

track_interest_sub_df = track_interest_df[['purchased_at', 'subscription_id']].copy()
track_interest_sub_df["purchased_at"] = track_interest_sub_df["purchased_at"].astype(str)
track_interest_sub_df["purchased_at"] = track_interest_sub_df["purchased_at"].apply(cut7)
track_interest_sub_df["purchased_at"] = track_interest_sub_df["purchased_at"].apply(format_date2)

all_data_df = pd.merge(track_interest_sub_df, subscriptions_sub_df, on="subscription_id", how="outer")
all_data_df['purchased_at'].fillna(0, inplace=True)
all_data_df = all_data_df[all_data_df.purchased_at != 0]

# *** NEW SUBSCRIBERS OVER TIME *** 

new_subs_df = pd.DataFrame(all_data_df['purchased_at'])
new_subs_df = new_subs_df.rename(columns={'purchased_at': 'Date Purchased'})
new_subs_df = new_subs_df.dropna(how = 'all')
num_new_subs = new_subs_df.groupby(['Date Purchased']).size().to_frame(name = 'Number of Subscribers').reset_index()

# Add a new column, 'Number of Subscribers From Trialers'. Initialize entire column to 0, preparatory to merge.
 
num_new_subs['Number of Subscribers (Trialers)'] = 0

# **** NEW TRIALERS OVER TIME ***

new_subs_from_trial_df = all_data_df[['purchased_at', 'was_trial']].copy()
new_subs_from_trial_df = new_subs_from_trial_df.dropna()
new_subs_from_trial_df = new_subs_from_trial_df[new_subs_from_trial_df.was_trial == "t"]

# Delete the 'was_trial' column, currently holding trues-only, and build a "trues-only" frequency
# table.
  
del new_subs_from_trial_df["was_trial"]
new_subs_from_trial_df = new_subs_from_trial_df.rename(columns={'purchased_at': 'Date Purchased'})
num_new_subs_from_trial = new_subs_from_trial_df.groupby(['Date Purchased']).size().to_frame(name = 'Number of Subscribers (Trialers)').reset_index()

# *** COMBINED: NEW SUBSCRIPTIONS AND TRIALS OVER TIME ***

subscribers_over_time_trialers_overlayed = pd.merge(num_new_subs, num_new_subs_from_trial, on="Date Purchased", how="outer")

# There'll two columns with the lede, "Number of Subscribers (Trialers)..."

del subscribers_over_time_trialers_overlayed["Number of Subscribers (Trialers)_x"]
subscribers_over_time_trialers_overlayed = subscribers_over_time_trialers_overlayed.rename(columns={'Number of Subscribers (Trialers)_y': 'Number of Subscribers (Trialers)'})
subscribers_over_time_trialers_overlayed['Number of Subscribers (Trialers)'].fillna(0, inplace=True)
subscribers_over_time_trialers_overlayed["Date Purchased"] = subscribers_over_time_trialers_overlayed["Date Purchased"].apply(date_converter)

# Take out the placeholder 9999-12-31

subscribers_over_time_trialers_overlayed = subscribers_over_time_trialers_overlayed[:-1]
""""


#  Plot Data

df = subscribers_over_time_trialers_overlayed
fig = make_subplots(rows=1, cols=1, subplot_titles=(""))
fig.add_trace(go.Scatter(x=df["Date Purchased"], y=df["Number of Subscribers"], name="Daily New Subscriptions, Daily"),
                         row=1, col=1)
fig.add_trace(go.Scatter(x=df["Date Purchased"], y=df["Number of Subscribers (Trialers)"], name="Daily New Subscriptions (Trials)"), 
                         row=1, col=1)

fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_yaxes(title_text="Number", row=1, col=1)
fig.update_layout(title_text="Per Day New Subscriptions | Trials Overlayed")

fig.show()

"""
#fig.write_html('index.html', auto_open=True)
"""

"\n#fig.write_html('index.html', auto_open=True)\n"

In [64]:
# **** ACTIVE PAID SUBSCRIPTIONS OVER TIME ***
"""
all_paid = all_data_df

# We need two separate views/tables to achieve the aim. This is one.
all_paid_grped_sub_df = all_paid.groupby(['purchased_at']).size().to_frame(name="No. of Paid Subscriptions").reset_index()

all_paid_grped_sub_df["Active Paid Subscriptions, Daily"] = 0
all_paid_grped_sub_df["Active Paid Subscriptions, Daily (w/o Trials)"] = 0
all_paid_grped_sub_df["Active Trialers, Daily"] = 0
all_paid_grped_sub_df["Active DNRs"] = 0
all_paid_grped_by_purchased_at = all_paid.groupby(['purchased_at'])

#Convert SeriesGroupBy to DataFrame and reformat date columns
all_paid_grped_by_purchased_at = all_paid_grped_by_purchased_at.apply(pd.DataFrame)
all_paid_grped_by_purchased_at['purchased_at'] = all_paid_grped_by_purchased_at['purchased_at'].astype(str)
all_paid_grped_by_purchased_at['purchased_at'] = all_paid_grped_by_purchased_at['purchased_at'].apply(date_converter)
all_paid_grped_by_purchased_at['paid_until'] = all_paid_grped_by_purchased_at['paid_until'].astype(str)
all_paid_grped_by_purchased_at["paid_until"] = all_paid_grped_by_purchased_at["paid_until"].apply(cut7)
all_paid_grped_by_purchased_at['paid_until'] = all_paid_grped_by_purchased_at['paid_until'].apply(date_converter)

print(all_paid_grped_by_purchased_at, "\n")

limit = len(all_paid_grped_sub_df.index)
for i in range(limit):
    this_purchased_at = all_paid_grped_sub_df['purchased_at'][i]

    # compare each purchased_at with ENTIRE HISTORY of paid_untils up until the purchased_at date

    subs_b4_this_purchased_at_date = all_paid_grped_by_purchased_at[all_paid_grped_by_purchased_at.purchased_at < date.fromisoformat(this_purchased_at)]

    active_paid_subs_b4_this_purchased_at_date = subs_b4_this_purchased_at_date[subs_b4_this_purchased_at_date.paid_until > date.fromisoformat(this_purchased_at)]
    active_paid_subs_b4_this_purchased_at_date_SAN_TRIAL = active_paid_subs_b4_this_purchased_at_date[active_paid_subs_b4_this_purchased_at_date.was_trial == "f"]

    # **** ACTIVE DNRs % ****
    active_paid_subs_b4_this_purchased_at_date_ACTIVE_DNRs = active_paid_subs_b4_this_purchased_at_date[active_paid_subs_b4_this_purchased_at_date.do_not_renew == "t"]
    
    # **** ACTIVE TRIALERS OVER TIME ****
    active_paid_subs_b4_this_purchased_at_date_TRIAL = active_paid_subs_b4_this_purchased_at_date[active_paid_subs_b4_this_purchased_at_date.was_trial == "t"]
    #print("==> ", this_purchased_at, "\n", active_paid_subs_b4_this_purchased_at_date.count()["paid_until"],"\n")
    
    all_paid_grped_sub_df["Active Paid Subscriptions, Daily"][i] = active_paid_subs_b4_this_purchased_at_date.count()["paid_until"]
    all_paid_grped_sub_df["Active Paid Subscriptions, Daily (w/o Trials)"][i] = active_paid_subs_b4_this_purchased_at_date_SAN_TRIAL.count()["paid_until"]
    all_paid_grped_sub_df["Active Trialers, Daily"][i] = active_paid_subs_b4_this_purchased_at_date_TRIAL.count()["paid_until"]
    all_paid_grped_sub_df["Active DNRs"][i] = active_paid_subs_b4_this_purchased_at_date_ACTIVE_DNRs.count()['paid_until']

# Note: Delete last row because of placeholder purchased_at  date "9999-12-13"
all_paid_grped_sub_df = all_paid_grped_sub_df[:-1]
all_paid_grped_sub_df = all_paid_grped_sub_df.rename(columns={'purchased_at': 'Date Purchased'})
"""

print("**********************************************************************\n\n", all_paid_grped_sub_df, "\n")

#  Plot Data

df = all_paid_grped_sub_df
fig = make_subplots(rows=2, cols=2, subplot_titles=(""))

fig.add_trace(go.Scatter(x=df["Date Purchased"], y=df["Active Paid Subscriptions, Daily"], name="Active Paid Subscriptions, Daily"), 
                         row=1, col=1)
fig.add_trace(go.Scatter(x=df["Date Purchased"], y=df["Active Paid Subscriptions, Daily (w/o Trials)"], name="Active Paid Subscriptions, Daily (w/o Trials)"), 
                         row=1, col=1)

fig.add_trace(go.Scatter(x=df["Date Purchased"], y=df["Active Trialers, Daily"], name="Active Trialers, Daily"), row=1, col=1)

fig.add_trace(go.Scatter(x=df["Date Purchased"], y=100.0 * (df["Active DNRs"]/df["Active Paid Subscriptions, Daily"]), name="Active DNRs Percentages"), row=2, col=1)

fig.add_trace(go.Scatter(x=df["Date Purchased"], y=100.0 * (df["Active Trialers, Daily"]/df["Active Paid Subscriptions, Daily"]), name="Active Trialers Percentages"), row=1, col=2)

fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_yaxes(title_text="Number", row=1, col=1)

fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Percentage", row=2, col=1)

fig.update_xaxes(title_text="Date", row=1, col=2)
fig.update_yaxes(title_text="Percentage", row=1, col=2)

fig.update_layout(title_text="Active Paid Subscriptions, Daily (All) && Active Trialers, Daily")

fig.show()

      purchased_at  subscription_id                     stripe_id  paid_until  \
0       9999-12-31              NaN                           NaN  9999-12-31   
1       9999-12-31              NaN                           NaN  9999-12-31   
2       9999-12-31              NaN                           NaN  9999-12-31   
3       9999-12-31              NaN                           NaN  9999-12-31   
4       9999-12-31              NaN                           NaN  9999-12-31   
...            ...              ...                           ...         ...   
54820   2022-02-27          12059.0  sub_1KXxDtJMuPhqEMcip5MmHeeu  2022-03-27   
54821   9999-12-31          12061.0                           NaN  9999-12-31   
54822   2022-02-27          12062.0  sub_1KXxXKJMuPhqEMciItZzEpnK  2022-03-27   
54823   2022-02-27          12060.0  sub_1KXxOiJMuPhqEMci5dg4yUHt  2022-03-02   
54824   9999-12-31          12063.0                           NaN  9999-12-31   

      was_trial do_not_rene

<ipython-input-64-f733a8d015e3>:42: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-64-f733a8d015e3>:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-64-f733a8d015e3>:44: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-64-f733a8d015e3>:45: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

**********************************************************************

     Date Purchased  No. of Paid Subscriptions  \
0       2021-10-14                          4   
1       2021-10-15                          5   
2       2021-10-16                          1   
3       2021-10-17                          2   
4       2021-10-18                          1   
..             ...                        ...   
132     2022-02-23                        139   
133     2022-02-24                         87   
134     2022-02-25                        126   
135     2022-02-26                        172   
136     2022-02-27                         69   

     Active Paid Subscriptions, Daily  \
0                                   0   
1                                   4   
2                                   9   
3                                  10   
4                                  12   
..                                ...   
132                              1862   
133       

In [65]:
new_subs_set_DNR_df = all_data_df[['purchased_at', 'do_not_renew']].copy()
new_subs_set_DNR_df = new_subs_set_DNR_df.dropna()
new_subs_set_DNR_df = new_subs_set_DNR_df[new_subs_set_DNR_df.do_not_renew == "t"]
  
del new_subs_set_DNR_df["do_not_renew"]
new_subs_set_DNR_df = new_subs_set_DNR_df.rename(columns={'purchased_at': 'Date Purchased'})
new_subs_set_DNR_df = new_subs_set_DNR_df.groupby(['Date Purchased']).size().to_frame(name = 'Number of Subscribers (DNRed)').reset_index()

# *** COMBINED: NEW SUBSCRIPTIONS AND DNRed OVER TIME ***

del num_new_subs["Number of Subscribers (Trialers)"] # leftover from previous graphing efforts
subscribers_over_time_DNRed_overlayed = pd.merge(num_new_subs, new_subs_set_DNR_df, on="Date Purchased", how="outer")
subscribers_over_time_DNRed_overlayed['Number of Subscribers (DNRed)'].fillna(0, inplace=True)
subscribers_over_time_DNRed_overlayed["Date Purchased"] = subscribers_over_time_DNRed_overlayed["Date Purchased"].apply(date_converter)
subscribers_over_time_DNRed_overlayed = subscribers_over_time_DNRed_overlayed[:-1]

#  Plot Data

df = subscribers_over_time_DNRed_overlayed
fig = make_subplots(rows=1, cols=1, subplot_titles=(""))
fig.add_trace(go.Scatter(x=df["Date Purchased"], y=df["Number of Subscribers"], name="New Subscriptions, Daily"),
                         row=1, col=1)
fig.add_trace(go.Scatter(x=df["Date Purchased"], y=df["Number of Subscribers (DNRed)"], name="Subscriptions Commencing with DNR=T, Daily"), 
                         row=1, col=1)
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_yaxes(title_text="Number", row=1, col=1)
fig.update_layout(title_text="Subscriptions Commencing With DNR \"True\"")

fig.show()

In [66]:
new_subs_status7_df = all_data_df[['purchased_at', 'status']].copy()
new_subs_status7_df = new_subs_status7_df.dropna()
new_subs_status7_df = new_subs_status7_df[new_subs_status7_df.status == 7]

del new_subs_status7_df["status"]
new_subs_status7_df = new_subs_status7_df.rename(columns={'purchased_at': 'Date Purchased'})
new_subs_status7_df = new_subs_status7_df.groupby(['Date Purchased']).size().to_frame(name = 'Number of Subscriptions (Cancelled)').reset_index()

# *** COMBINED: NEW SUBSCRIPTIONS AND CANCELLED OVER TIME ***

subscribers_over_time_cancelled_overlayed = pd.merge(num_new_subs, new_subs_status7_df, on="Date Purchased", how="outer")
subscribers_over_time_cancelled_overlayed['Number of Subscriptions (Cancelled)'].fillna(0, inplace=True)
subscribers_over_time_cancelled_overlayed["Date Purchased"] = subscribers_over_time_cancelled_overlayed["Date Purchased"].apply(date_converter)
subscribers_over_time_cancelled_overlayed = subscribers_over_time_cancelled_overlayed[:-1]

#  Plot Data

df = subscribers_over_time_cancelled_overlayed
fig = make_subplots(rows=1, cols=1, subplot_titles=(""))
fig.add_trace(go.Scatter(x=df["Date Purchased"], y=df["Number of Subscribers"], name="New Subscriptions, Daily"),
                         row=1, col=1)
fig.add_trace(go.Scatter(x=df["Date Purchased"], y=df["Number of Subscriptions (Cancelled)"], name="Per Day No. of Subscriptions With Status \"Cancelled\""), 
                         row=1, col=1)
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_yaxes(title_text="Number", row=1, col=1)
fig.update_layout(title_text="Per Day No. of Subscriptions With Status 7, \"Cancelled\"")

fig.show()